In [2]:
import sys
sys.path.append("../")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import requests
import json
import time

from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [5]:
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
!curl -d "{\"yandexPassportOauthToken\":\"y0_AgAAAAAFAhuyAATuwQAAAADTA2uLt9P4wIhIRcycxMNG36QNKa9u_pc\"}" "https://iam.api.cloud.yandex.net/iam/v1/tokens" --ssl-no-revoke -o iamToken.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   343  100   254  100    89    888    311 --:--:-- --:--:-- --:--:--  1199


In [7]:
import json
with open('iamToken.txt') as f:
    d = json.loads(f.read())
    IAM_TOKEN = d['iamToken']

In [8]:
# IAM_TOKEN = 't1.9euelZqaz5DGjI6Py5bLnoqTlo_Oju3rnpWaj46Nkc6ejZaLyZvOmZqWj53l8_dIF2Zk-e9eDic9_t3z9whGY2T5714OJz3-.1UJE5innTD10cDR2-7YbCzvwem5VKOvoIQCksQwLbJAA7oAdm4OpX9lgQyC6cosbkLvAMqxKaFT1SfQlDXTlBg'
folder_id = 'b1grvnc3e8sgtm7qcsja'
IAM_TOKEN

't1.9euelZrKk8iak5mRnY-VypeNmJGRk-3rnpWam46Pjp6Qj5uam5KQxo_Pxo3l8_dcfH1i-e8VAFRY_N3z9xwre2L57xUAVFj8.ho5XQi-7__NRGLFjv2LysDukZTgeYfmkChfUvivdOhU_WPSOBJpa4awo60QUmurc-nYz8apVJHQpk6TGsA3GDA'

In [9]:
# docker run --rm -p 3333:3333 inemo/isanlp
# docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

In [11]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault  
#
ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
res = ppl('Мы поехали на дачу.')


In [12]:
%%time
languages = ['tat', 'kaz', 'kir', 'bak', 'uzb'] # языки татарский, казахский, киргизский
language = languages[0]
url = 'https://beta.apertium.org/index.eng.html#analysis?aLang=' + language + '&aQ='    
browser = webdriver.Chrome()
browser.get(url)


CPU times: total: 31.2 ms
Wall time: 4.26 s


In [14]:
class Model_srl2:
    def __init__(self, RUS_SENTENCE=None):
        self.RUS_SENTENCE = RUS_SENTENCE
        self.ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
        self.language_code_dict =  {
        'татарский': ['tt', 'Tatar'],
        'казахский' : ['kk', 'Kazakh'],
        'башкирский': ['ba', 'Bashqort'],
        'киргизский' : ['ky', 'Kyrgyz']
        }

    def translation(self, texts, source_language='tt', target_language='ru'):
        #print('texts', texts)
        body = {
            "targetLanguageCode": target_language,
            "texts": texts,
            "folderId": folder_id,
            "sourceLanguageCode":  source_language
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {0}".format(IAM_TOKEN)
        }
        url = 'https://translate.api.cloud.yandex.net/translate/v2/translate'
        response = requests.post(url,
            json = body,
            headers = headers
        )
        if response.status_code != 200:
            print('Ожидаю 0.5 секунды...')  
            time.sleep(0.5)
            response = requests.post(url,
            json = body,
            headers = headers
            )
        d = json.loads(response.text)
        #print(texts, d)
        translations = d['translations']
#         print([t['text'] for t in translations])
        return [t['text'] for t in translations]
    
    def drop_punkt(self, word):
        punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
        for p in punc:
            word = word.replace(p,'')
        return word
    
    def tokenize(self, text):
        words = text.split()
        return [self.drop_punkt(word) for word in words]
    
    def get_morph(self, text, language):
        lang_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[1]/div/select')
        lang_field.send_keys(language)

        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea').clear()
        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea')

        text = self.drop_punkt(text)
        text_field.send_keys(text)

        button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[3]/div/button')
        button.click()
        sleep(1)

        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        translation = soup.find_all('td',class_="text-left")
        words = []
        for word in translation:
            word = word.text.split()
            for i, w in enumerate(word):
                if w == '↤':
                    break
            word = word[:i]
            for w in word:
                words.append(w)
        return words


    def print_roles(self, lemma, role_annot, targetLanguage=None):
        roles = []
        for sent_num, ann_sent in enumerate(role_annot):
            word_role = {}
            
            for event in ann_sent:
#                 print(event.pred)
                lemma_pred = lemma[sent_num][event.pred[0]]
#                 print(lemma_pred)
                if targetLanguage is not None:
                    trg_lang = self.language_code_dict[targetLanguage][0]
                    src_lang = 'ru'
                    #print('=====Pred: {}'.format(self.translation(lemma_pred, src_lang, trg_lang)[0]))
                    word_role['V'] = self.translation(lemma_pred, src_lang, trg_lang)
                else:
                    print('=====Pred: {}'.format(lemma_pred))
                for arg in event.args:
                    lemma_arg  = lemma[sent_num][arg.begin]
                    if targetLanguage is not None:
                        #print('Arg({}): {}'.format(arg.tag, self.translation(lemma_arg, src_lang, trg_lang)[0]))
                        word_role['Arg({})'.format(arg.tag)] = self.translation(lemma_arg, src_lang, trg_lang)
                    else:
                        print('Arg({}): {}'.format(arg.tag, lemma_arg))
            roles.append(word_role)
        return roles
    
    def get_srl_turkic(self, rus_text, selected_language):
        res = ppl(rus_text)
        return self.print_roles(res['lemma'], res['srl'], selected_language)
        
    def get_tags(self, words, srl_verb):
        new_tags = []
        for word in words:
            for srl in srl_verb.keys(): 
                if self.drop_punkt(word) in srl_verb[srl]:
                    role = srl
                    break
                else:
                    role = 'O'
            new_tags.append(role)
        return new_tags

    def get_new_roles(self, tat_sentence, morph_sentence, roles, words):
        result = {'verbs': []}
        metric = []
        for verb in roles:
            new_roles = {}
            srl_verb = {'verb':''}
#             print(verb)
            for srl in verb.keys():
    #             morph = get_morph(verb[srl], language)[0]
                morph = verb[srl][0]
#                 print(morph)
        #         Ищем слово в предложении по морфеме
                for i in range(len(morph), 0, -1):
#                     print(i, morph[:i] in morph_sentence,morph[:i], morph_sentence)
                    if morph[:i] in morph_sentence:
                        idx = morph_sentence.index(morph[:i])
                        new_roles[srl] = words[idx]
                        if srl == 'V':
                            srl_verb['verb'] = words[idx]
                        break


        #     Считаем сколько слов правильно перевели

            srl_verb['description'] = new_roles

            srl_verb['tags'] = self.get_tags(words, new_roles)

            metric.append(len(new_roles.keys()) / len(verb.keys()))
            result['verbs'].append(srl_verb)

        result['words'] = words
        result['metric'] = round(np.mean(metric),2)
        return result


    def predict(self, tat_sentence, language_ru_name, tries=5):
        pred = None
        language = self.language_code_dict[language_ru_name]

        #перевод
        rus_sentence = self.translation(tat_sentence, source_language=language[0])[0]
#         print(rus_sentence)
        # стемминг слов исходного предложения
        stem_input_words = self.get_morph(tat_sentence, language[1])
        # получить список словарей лемма - роль
        roles = self.get_srl_turkic(rus_sentence, language_ru_name)
        # Список слов в предложении
        words = tat_sentence.split()

        for i in range(tries):
            try:
                pred = self.get_new_roles(tat_sentence, stem_input_words, roles, words)
                break
            except:
                continue
        return pred
    


In [15]:
srl = Model_srl2()
with open('text_tt.txt', encoding='utf-8') as f:
    texts_tt = f.readlines()
selected_language = 'татарский'
result = srl.predict(texts_tt[2], selected_language)
result

{'verbs': [{'verb': 'юды.',
   'description': {'V': 'юды.', 'Arg(агенс)': 'Әни'},
   'tags': ['Arg(агенс)', 'O', 'V', 'O', 'O', 'O']},
  {'verb': 'йөртте.',
   'description': {'V': 'йөртте.', 'Arg(агенс)': 'Әти'},
   'tags': ['O', 'O', 'O', 'Arg(агенс)', 'O', 'V']}],
 'words': ['Әни', 'рамны', 'юды.', 'Әти', 'машинаны', 'йөртте.'],
 'metric': 0.83}

In [16]:
roles = [{'V': 'юу', 'Arg(агенс)': 'әни', 'Arg(пациенс)': 'кыса'},
        {'V': 'йөртү', 'Arg(агенс)': 'әти'}]

# Validation 

In [20]:
import pandas as pd

In [31]:
df = pd.read_csv('dataset_news-commentary-v14-wmt19.en-kk_full.csv', sep='\t')

In [35]:
srl = Model_srl2()
selected_language = 'казахский'
res = []
#  Проверка что все работает
result = srl.predict(df.loc[1,'kk'], selected_language)
result

In [36]:
for i in tqdm(range(0, df.shape[0])):
    srl_labeled = srl.predict(df.loc[i,'kk'], selected_language)
    res.append(srl_labeled)
    if i % 500 == 0:
        df_res = pd.DataFrame(res)
        df_res.to_csv('kk_res.csv', index=False)


C:\Users\rukis\AppData\Local\Temp\ipykernel_1240\2579236067.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, df.shape[0])):


  0%|          | 0/31 [00:00<?, ?it/s]

In [37]:
metrics = []
for i in range(len(res)):
    if df_labeled.loc[i,0] is not None:
        metrics.append(df_labeled.loc[i,0]['metric'])

NameError: name 'df_labeled' is not defined

In [ ]:
np.mean(metrics)

In [ ]:
len(res)